# CSE 252B: Computer Vision II, Winter 2019 – Assignment 3
### Instructor: Ben Ochoa
### Due: Wednesday, February 19, 2020, 11:59 PM

## Instructions
* Review the academic integrity and collaboration policies on the course website.
* This assignment must be completed individually.
* All solutions must be written in this notebook
* Math problems must be done in Markdown/LATEX.
* You must show your work and describe your solution.
* Programming aspects of this assignment must be completed using Python in this notebook.
* Your code should be well written with sufficient comments to understand, but there is no need to write extra markdown to describe your solution if it is not explictly asked for.
* This notebook contains skeleton code, which should not be modified (This is important for standardization to facilate effecient grading).
* You may use python packages for basic linear algebra, but you may not use packages that directly solve the problem. If you are uncertain about using a specific package, then please ask the instructional staff whether or not it is allowable.
* You must submit this notebook exported as a pdf. You must also submit this notebook as an .ipynb file.
* Your code and results should remain inline in the pdf (Do not move your code to an appendix).
* You must submit both files (.pdf and .ipynb) on Gradescope. You must mark each problem on Gradescope in the pdf.
* It is highly recommended that you begin working on this assignment early.

## Problem 1 (Programming):  Estimation of the Camera Pose - Outlier rejection (20 points)
  Download input data from the course website.  The file
  hw3_points3D.txt contains the coordinates of 60 scene points
  in 3D (each line of the file gives the $\tilde{X}_i$, $\tilde{Y}_i$,
  and $\tilde{Z}_i$ inhomogeneous coordinates of a point).  The file
  hw3_points2D.txt contains the coordinates of the 60
  corresponding image points in 2D (each line of the file gives the
  $\tilde{x}_i$ and $\tilde{y}_i$ inhomogeneous coordinates of a
  point).  The corresponding 3D scene and 2D image points contain both
  inlier and outlier correspondences.  For the inlier correspondences,
  the scene points have been randomly generated and projected to image
  points under a camera projection matrix (i.e., $\boldsymbol{x}_i = \boldsymbol{P}
  \boldsymbol{X}_i$), then noise has been added to the image point
  coordinates.

  The camera calibration matrix was calculated for a $1280 \times 720$
  sensor and $45\,^\circ$ horizontal field of view lens.  The
  resulting camera calibration matrix is given by
  
  $\boldsymbol{K} = \left[
    \begin{array}{c c c}
      1545.0966799187809 & 0 & 639.5\\
      0 & 1545.0966799187809 & 359.5\\
      0 & 0 & 1
    \end{array}\right]$
    
  For each image point $\boldsymbol{x} = (x, y, w)^\top = (\tilde{x},
  \tilde{y}, 1)^\top$, calculate the point in normalized coordinates
  $\hat{\boldsymbol{x}} = \boldsymbol{K}^{-1} \boldsymbol{x}$.

  Determine the set of inlier point correspondences using the
  M-estimator Sample Consensus (MSAC) algorithm, where the maximum
  number of attempts to find a consensus set is determined adaptively.
  For each trial, use the 3-point algorithm of Finsterwalder (as
  described in the paper by Haralick et al.) to estimate the camera
  pose (i.e., the rotation $\boldsymbol{R}$ and translation $\boldsymbol{t}$ from the
  world coordinate frame to the camera coordinate frame), resulting in
  up to 4 solutions, and calculate the error and cost for each
  solution.  Note that the 3-point algorithm requires the 2D points in
  normalized coordinates, not in image coordinates.  Calculate the
  projection error, which is the (squared) distance between projected
  points (the points in 3D projected under the normalized camera
  projection matrix $\hat{\boldsymbol{P}} = [\boldsymbol{R} | \boldsymbol{t}]$) and the
  measured points in normalized coordinates (hint: the error tolerance
  is simpler to calculate in image coordinates using $\boldsymbol{P} =
  \boldsymbol{K} [\boldsymbol{R} | \boldsymbol{t}]$ than in normalized coordinates using
  $\hat{\boldsymbol{P}} = [\boldsymbol{R} | \boldsymbol{t}]$).

  
  Hint: this problem has codimension 2. 
 

#### Report your values for:
 * the probability $p$ that as least one of the random samples does not contain any outliers
 * the probability $\alpha$ that a given point is an inlier
 * the resulting number of inliers
 * the number of attempts to find the consensus set

In [600]:
import numpy as np
import time

def Homogenize(x):
    # converts points from inhomogeneous to homogeneous coordinates
    return np.vstack((x,np.ones((1,x.shape[1]))))

def Dehomogenize(x):
    # converts points from homogeneous to inhomogeneous coordinates
    return x[:-1]/x[-1]

    
# load data
x0=np.loadtxt('hw3_points2D.txt').T
X0=np.loadtxt('hw3_points3D.txt').T
print('x is', x0.shape)
print('X is', X0.shape)

K = np.array([[1545.0966799187809, 0, 639.5], 
      [0, 1545.0966799187809, 359.5], 
      [0, 0, 1]])

print('K =')
print(K)
    
def ComputeCost(P, x, X, K):
    # Inputs:
    #    P - camera projection matrix
    #    x - 2D groundtruth image points
    #    X - 3D groundtruth scene points
    #    K - camera calibration matrix
    #
    # Output:
    #    cost - total projection error
    """your code here"""
    n = x.shape[1]
    covarx = np.eye(2*n) # covariance propagation
    cost = 0
    x_pred = Dehomogenize(np.dot(np.dot(K,P),Homogenize(X)))
    x_error = x_pred - x
    epslon = x_error.reshape((2*n,1), order = 'F')
    x_cov = np.cov(x)
    cost = np.dot(epslon.T, np.dot(np.linalg.inv(covarx), epslon))
    return cost[0][0]



x is (2, 60)
X is (3, 60)
K =
[[1.54509668e+03 0.00000000e+00 6.39500000e+02]
 [0.00000000e+00 1.54509668e+03 3.59500000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [452]:
from scipy.stats import chi2
def distance(x1,x2):
    d = [(x1[i] - x2[i])**2 for i in range(len(x1))]
    #print("d:{}".format(sum(d)))
    return sum(d)[0]**0.5

def units(x,f):
    '''
    input:
        x: normalized coordinate
    return:
        (u,v)
        j: unit vector
    '''
    u = f * x[0,0]/x[2,0]
    v = f * x[1,0]/x[2,0]
    j = 1/np.sqrt(u**2+v**2+f**2) * np.array([u,v,f])
    return (u,v),j.T

def sign(x):
    if x < 0:
        return -1
    elif x > 0:
        return 1
    else:
        return 0   
    
def append_UV(m, n, parameters,has_solution):
    #if B**2 - AC > 0: append solution to U, V
    U, V, cos_gamma, cos_beta, a, b, c = parameters 

    A = b**2 - m**2*c**2
    B = c**2*(cos_beta-n)*m - b**2*cos_gamma
    C = -c**2*n**2 + 2*c**2*n*cos_beta+b**2-c**2
    if B**2 - A*C >= 0 :
        u_large = - (sign(B)/A) * (abs(B) + math.sqrt(B**2 - A*C))
        u_small = C / (A*u_large)
        v_large = u_large*m + n
        v_small = u_small*m + n
        U.extend([u_large,u_small])
        V.extend([v_large,v_small])

def GHIJ(a, b, c, cos_alpha, cos_beta, cos_gamma):
    sin_alpha = (1 - cos_alpha**2)**0.5
    sin_beta = (1 - cos_beta**2)**0.5
    sin_gamma = (1 - cos_gamma**2)**0.5
    G = c**2 * (c**2 * sin_beta**2 - b**2 * sin_gamma**2)
    H = b**2 *(b**2 - a**2)*sin_gamma**2 + c**2 * (c**2 + 2*a**2)*sin_beta**2 + 2*b**2*c**2 *(-1+ cos_alpha*cos_beta*cos_gamma)
    I = b**2*(b**2-c**2)*sin_alpha**2 + a**2*(a**2+2*c**2)*sin_beta**2 + 2*a**2*b**2*(-1+cos_alpha*cos_beta*cos_gamma)
    J = a**2*(a**2*sin_beta**2 - b**2*sin_alpha**2)
    return G,H,I,J
    
def ABCDEF(a, b, c, cos_alpha, cos_beta, cos_gamma, lamb_root):
    A = 1 + lamb_root
    B = -cos_alpha
    C = (b**2 - a**2)/(b**2) - lamb_root*(c**2/b**2)
    D = -lamb_root*cos_gamma
    E = (a**2/b**2 + lamb_root*c**2/b**2) * cos_beta
    F = -a**2/b**2 + lamb_root *(b**2-c**2)/b**2
    return A,B,C,D,E,F

def computeError(x,X,K,P):
    x_predict = Dehomogenize(np.dot(np.dot(K,P),Homogenize(X)))
    #print("x_predict.shape:{}".format(x_predict.shape))
    #print("x.shape:{}".format(x.shape))
    x_error = x_predict - x
    return np.linalg.norm(x_error)

    
def Finsterwalder(random_idx,f,K,x,X):
    # random_idx = list of 3 elenets
    
    P_best = np.zeros((3,4))
    camera_center = np.zeros([0,0,0])
    idx1,idx2,idx3 = random_idx
    X1,X2,X3 = X[:,idx1:idx1+1], X[:,idx2:idx2+1], X[:,idx3:idx3+1]
    x1,x2,x3 = x[:,idx1:idx1+1], x[:,idx2:idx2+1], x[:,idx3:idx3+1]

    a = distance(X2,X3)
    b = distance(X1,X3)
    c = distance(X1,X2)
    
    x1_normed = np.dot(np.linalg.inv(K), Homogenize(x1))
    x2_normed = np.dot(np.linalg.inv(K), Homogenize(x2))
    x3_normed = np.dot(np.linalg.inv(K), Homogenize(x3))
    
    (u1,v1),j1 =  units(x1_normed,f)
    (u2,v2),j2 =  units(x2_normed,f)
    (u3,v3),j3 =  units(x3_normed,f)
    
    cos_alpha = np.dot(j2,j3)
    cos_beta = np.dot(j1,j3)
    cos_gamma = np.dot(j1,j2)
    
    G,H,I,J = GHIJ(a, b, c, cos_alpha, cos_beta, cos_gamma)
    
    lambs = np.roots([G,H,I,J])
    check_real = np.isreal(lambs)
    lamb_root = 0
    for i in range(len(lambs)):
        if check_real[i]:
            lamb_root = np.real(lambs[i])

    A,B,C,D,E,F = ABCDEF(a, b, c, cos_alpha, cos_beta, cos_gamma, lamb_root)
    U,V = [],[]
    has_solution = False
    if B**2 - A*C >= 0 and E**2 - C*F >=0:
        p = math.sqrt(B**2-A*C)
        q = sign(B*E-C*D)*math.sqrt(E**2-C*F)
        m1 = (-B+p)/C
        m2 = (-B-p)/C
        n1 = -(E-q)/C
        n2 = -(E+q)/C
        parameters = [U, V, cos_gamma, cos_beta, a, b, c]
        append_UV(m1, n1, parameters, has_solution)
        append_UV(m2, n2, parameters, has_solution)
        
    if not U and not V:
        return P_best, has_solution
    has_solution = True
    X_list = [X1,X2,X3]
    cost_min = float("inf")

    for i in range(len(U)):
        u = U[i]
        v = V[i]
        s1 = math.sqrt(a**2 / (u**2+ v**2-2*u*v*cos_alpha))
        s2 = u*s1
        s3 = v*s1
        p1 = s1*j1
        p2 = s2*j2
        p3 = s3*j3
        p_list = [p1,p2,p3]
        X_mean = (X1+X2+X3) / 3
        p_mean = (p1+p2+p3) / 3
        
        X_hat = [X_list[i] - X_mean for i in range(len(X_list))]
        p_hat = [p_list[i] - p_mean for i in range(len(p_list))]
        BB , CC = np.zeros((3,3)), np.zeros((3,3))
        for i in range(3):
            BB[i,:] = X_hat[i].reshape((1,3))
            CC[i,:] = p_hat[i].reshape((1,3))
        S = np.dot(np.transpose(CC), BB)
        #print("S:{}".format(S))
        R,t = np.zeros((3,3)), np.zeros((3,1))
        _u, _, _vh = np.linalg.svd(S)
        if np.linalg.det(_u)* np.linalg.det(_vh.T) < 0:
            R = np.dot(np.dot(_u,np.diag([1,1,-1])),_vh)
        else:
            R = np.dot(_u,_vh)

        t =  p_mean.reshape((3,1)) - np.dot(R,X_mean)
        t.reshape((3,1))
        P = np.hstack((R,t))
        
        x_pred = np.dot(np.dot(K,P),Homogenize(X))
        x_pred = Dehomogenize(x_pred)
        cost = 0
        
        for i in range(x.shape[1]):
            cost += computeError(x[:,i:i+1],X[:,i:i+1],K,P)

        if cost < cost_min:
            cost_min = cost
            P_best = P
        
    return P_best, has_solution



In [1]:
def MSAC(x, X, K, thresh, tol, p):
    # Inputs:
    #    x - 2D inhomogeneous image points
    #    X - 3D inhomogeneous scene points
    #    K - camera calibration matrix
    #    thresh - cost threshold
    #    tol - reprojection error tolerance 
    #    p - probability that as least one of the random samples does not contain any outliers   
    #
    # Output:
    #    consensus_min_cost - final cost from MSAC
    #    consensus_min_cost_model - camera projection matrix P
    #    inliers - list of indices of the inliers corresponding to input data
    #    trials - number of attempts taken to find consensus set
    
    """your code here"""
    
    trials = 0
    max_trials = np.inf
    consensus_min_cost = np.inf
    consensus_min_cost_model = np.zeros((3,4))
    inliers = []
    alpha = 0.95
    s = 3 # sample size
    variance = 1
    thresh = chi2.ppf(alpha, df=2) * variance**2
    #print("thresh:{}".format(thresh))
    while trials < max_trials and consensus_min_cost > thresh:
        inliers_cur = []
        random_idx = random.sample(range(x.shape[1]),3)
        P, has_solution = Finsterwalder(radom_idx,K[0,0],K,x,X)

        if not has_solution:
            continue
        cost = 0
        cnt_inliers = 0
        for i in range(x.shape[1]):
            error = computeError(x[:,i:i+1],X[:,i:i+1],K,P)
            if error < tol:
                inliers_cur.append(i)
                cnt_inliers += 1
                cost += error
            else:
                cost += tol

        if cost < consensus_min_cost:
            consensus_min_cost = cost
            consensus_min_cost_model = P
            w = cnt_inliers / x.shape[1]
            inliers = inliers_cur

            if w != 0:
                max_trials = math.log(1-p) / math.log(1-w**s)

        trials += 1
    #print("when tol = {}, # of inliers:{}".format(tol, len(inliers)))
    #print("E attempts:{}".format(trials))
    #print("inliers:{}".format(inliers))
    #print("minCost:{}".format(consensus_min_cost))
    #inliers = np.random.randint(0, 59, size=10)
    
    return consensus_min_cost, consensus_min_cost_model, inliers, trials


# MSAC parameters 
thresh = 100
tol = 60
p = 0.99
alpha = 0.95

tic=time.time()
cost_MSAC, P_MSAC, inliers, trials = MSAC(x0, X0, K, thresh, tol, p)
#print('tol:{},cost:{},inliers:{},trials:{}, P = '.format(tol,cost_MSAC,len(inliers), trials))
#print(P_MSAC)
# choose just the inliers
x = x0[:,inliers]
X = X0[:,inliers]

toc=time.time()
time_total=toc-tic

# display the results
print('took %f secs'%time_total)
# print('%d iterations'%trials)
# print('inlier count: ',len(inliers))
print('MSAC Cost=%.9f'%cost_MSAC)
print('P = ')
print(P_MSAC)
print('inliers: ',inliers)
print("# of inliers: ",len(inliers))
print("# attempts: ",trials)


NameError: name 'time' is not defined

#### Final values for parameters
* $p$ = 0.95
* $\alpha$ = 0.95
* tolerance = 60
* num_inliers = 47
* num_attempts = 8

## Problem 2 (Programming): Estimation of the Camera Pose - Linear Estimate (30 points)
  Estimate the normalized camera projection matrix
  $\hat{\boldsymbol{P}}_\text{linear} = [\boldsymbol{R}_\text{linear} |
  \boldsymbol{t}_\text{linear}]$ from the resulting set of inlier
  correspondences using the linear estimation method (based on the
  EPnP method) described in lecture. Report the resulting 
  $\boldsymbol{R}_\text{linear}$ and $\boldsymbol{t}_\text{linear}$.

In [603]:
def EPnP(x, X, K):
    # Inputs:
    #    x - 2D inlier points
    #    X - 3D inlier points
    # Output:
    #    P - normalized camera projection matrix
    
    """your code here"""
    
    x_mean_3d, y_mean_3d, z_mean_3d= np.mean(X, axis = 1)
    X_mean = np.mean(X, axis = 1).reshape((3,1))
    #print("X_mean:{}".format(X_mean))
    R = np.eye(3) 
    t = np.array([[0,0,0]]).T 
    P = np.concatenate((R, t), axis=1)
    X_cam_de = Dehomogenize(np.dot(np.linalg.inv(K),Homogenize(x)))
    X_cov = np.cov(X)
    X_sigma = np.trace(X_cov)
    u, _, vh = np.linalg.svd(X_cov)
    v = vh.T
    v1,v2,v3 = v[:,0:1],v[:,1:2],v[:,2:3]
    C1 = X_mean
    C2 = X_mean + v1
    C3 = X_mean + v2
    C4 = X_mean + v3
    A = np.hstack((np.hstack((C2-C1,C3-C1)),C4-C1))
    A_inv = np.linalg.inv(A)
    alphas = np.zeros((4,X.shape[1]))
    M = np.zeros((2*X.shape[1], 12))
    for i in range(X.shape[1]):
    #for i in range(10):
        b = X[:,i:i+1] - C1
        alpha = np.dot(A_inv, b)
        alpha2 ,alpha3, alpha4 = alpha
        alpha1 = 1 - alpha2- alpha3 - alpha4
        alphas[:,i] = np.array([alpha1,alpha2,alpha3,alpha4]).T
        xx = X_cam_de[0,i]
        yy = X_cam_de[1,i]
        sub_M = np.array([[alpha1, 0, -alpha1*xx, alpha2, 0, -alpha2*xx, alpha3, 0, -alpha3*xx, alpha4, 0, -alpha4*xx],
                          [0, alpha1, -alpha1*yy, 0, alpha2, -alpha2*yy, 0, alpha3, -alpha3*yy, 0, alpha4, -alpha4*yy]])
        M[2*i:2*i+2,:] = sub_M
    #print(M)
    _,_,vh = np.linalg.svd(M)
    v = vh[-1,:].reshape((12,1))
    C1_cam, C2_cam, C3_cam, C4_cam = v[:3,:],v[3:6,:],v[6:9,:],v[9:12,:]
    
    #print("C1_cam:{}, C2_cam:{}, C3_cam:{}, C4_cam:{}".format(C1_cam, C2_cam, C3_cam, C4_cam))
    control_points = np.hstack((np.hstack((np.hstack((C1_cam,C2_cam)),C3_cam)),C4_cam))
    
    X_cam = np.zeros(X.shape)
    for i in range(X.shape[1]):
        alpha = alphas[:,i:i+1]
        X_cam[:,i:i+1] = np.dot(control_points, alpha)
    X_cam_mean = np.mean(X_cam, axis = 1).reshape((3,1))
    X_cam_cov = np.cov(X_cam)
    X_cam_sigma = np.trace(X_cam_cov)
    
    BETA = math.sqrt(X_sigma/X_cam_sigma)
    if X_cam_mean[-1,0] < 0:
        BETA = -BETA
    X_cam = BETA * X_cam 
    X_cam_mean = np.mean(X_cam, axis = 1).reshape((3,1))
    X_cam_cov = np.cov(X_cam)
    X_cam_hat = X_cam - X_cam_mean
    X_hat = X -  X_mean
    B = X_hat.T
    C = X_cam_hat.T
    S = np.dot(C.T, B)
    u, D, vh = np.linalg.svd(S)
    if np.linalg.det(u) * np.linalg.det(vh.T) < 0:
        diag = np.diag([1,1,-1])
        R = np.dot(np.dot(u,diag), vh)
    else:
        R = np.dot(u,vh)
    t = X_cam_mean - np.dot(R,X_mean)
    P = np.concatenate((R, t), axis=1)

    return P

# Uncomment the following block to run EPnP on an example set of inliers.
# You MUST comment it before submitting, or you will lose points.
# The sample cost with these inliers should be around 57.85.
#x0=np.loadtxt('hw3_points2D.txt').T
#X0=np.loadtxt('hw3_points3D.txt').T
#inliers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]

x = x0[:,inliers]
X = X0[:,inliers]

tic=time.time()
P_linear = EPnP(x, X, K)
toc=time.time()
time_total=toc-tic

# display the results
print('took %f secs'%time_total)
print('R_linear = ')
print(P_linear[:,0:3])
print('t_linear = ')
print(P_linear[:,-1])
cost_linear = ComputeCost(P_linear, x, X, K)
print("cost:{}".format(cost_linear))

took 0.004337 secs
R_linear = 
[[ 0.28244697 -0.69008218  0.66634097]
 [ 0.67795091 -0.34784112 -0.64760259]
 [ 0.6786798   0.63465986  0.36959518]]
t_linear = 
[  5.89003956   9.34200202 175.49661172]
cost:6255.233593968516


## Problem 3 (Programming): Estimation of the Camera Pose - Nonlinear Estimate (30 points) 
  Use $\boldsymbol{R}_\text{linear}$ and $\boldsymbol{t}_\text{linear}$ as an
  initial estimate to an iterative estimation method, specifically the
  Levenberg-Marquardt algorithm, to determine the Maximum Likelihood
  estimate of the camera pose that minimizes the projection error
  under the normalized camera projection matrix $\hat{\boldsymbol{P}} =
  [\boldsymbol{R} | \boldsymbol{t}]$.  You must parameterize the camera rotation
  using the angle-axis representation $\boldsymbol{\omega}$ (where
  $[\boldsymbol{\omega}]_\times = \ln \boldsymbol{R}$) of a 3D rotation, which is
  a 3-vector.
  
  
  Report the initial cost (i.e. cost at iteration 0) and the cost at the end
  of each successive iteration. Show the numerical values for the final 
  estimate of the camera rotation $\boldsymbol{\omega}_\text{LM}$ and 
  $\boldsymbol{R}_\text{LM}$, and the camera translation $\boldsymbol{t}_\text{LM}$.

In [604]:
from scipy.linalg import block_diag, logm

# Note that np.sinc is different than defined in class
def Sinc(x):
    """your code here"""
    if x == 0:
        return 1
    else:
        return math.sin(x)/x
    
def dSinc(theta):
    if theta == 0:
        return 0
    return (math.cos(theta)/theta) - (math.sin(theta)/(theta**2))

def skew(w):
    # Returns the skew-symmetrix represenation of a vector
    """your code here"""
    n = w.shape[0]
    w_skew = np.zeros((n,n))
    w_skew[0,1] = -w[2,0]
    w_skew[0,2] = w[1,0]
    w_skew[1,2] = -w[0,0]
    w_skew[1,0] = w[2,0]
    w_skew[2,0] = -w[1,0]
    w_skew[2,1] = w[0,0]
    return w_skew

def Parameterize(R):
    # Parameterizes rotation matrix into its axis-angle representation
    """your code here"""
    R_log = logm(R)
    w1 = float(-R_log[1][2])
    w2 = float(R_log[0][2])
    w3 = float(R_log[1][0])
    w = np.array([[w1],[w2],[w3]])
    theta = np.linalg.norm(w)
    return w, theta

def Deparameterize(w):
    # Deparameterizes to get rotation matrix
    """your code here"""
    theta = np.linalg.norm(w)
    R = np.cos(theta) * np.identity(3) + Sinc(theta) * skew(w) + ((1 - math.cos(theta)) / (theta**2)) * w * w.T
    return R

def v_norm_de(v):
    return v.T / np.linalg.norm(v)


def ds_dtheta(theta):
    return (theta * math.sin(theta) - 2*(1-math.cos(theta)))/ (theta**3)

def cross(w, v):
    product = np.cross(w.T, v.T)
    return np.transpose(product)

def dXrotated_dw(theta, w,v):
    if theta < 1e-5:
        return skew(-v)
    dthe_dw = v_norm_de(v)
    p1 = Sinc(theta) * skew(-v)
    s = (1-math.cos(theta))/(theta**2)
    p2 = cross(w,v) * dSinc(theta) * dthe_dw
    p3 = cross(w, cross(w,v)) * ds_dtheta(theta) * dthe_dw
    tmp1 = np.dot(skew(w), skew(-v))
    tmp2 = skew(-cross(w,v))
    p4 = s*(tmp1 + tmp2)
    
    return p1+p2+p3+p4


def Jacobian(R, w, t, X):
    # compute the jacobian matrix
    # Inputs:
    #    R - 3x3 rotation matrix
    #    w - 3x1 axis-angle parameterization of R
    #    t - 3x1 translation vector
    #    X - 3D inlier points
    #
    # Output:
    #    J - Jacobian matrix of size 2*nx6
    
    """your code here"""

    J = np.zeros((2*X.shape[1],6))
    P = np.hstack((R,t))
    x_de = Dehomogenize(np.dot(P,Homogenize(X)))
    x_norm_inhom = Dehomogenize(np.dot(R, X) + t)
    theta = np.linalg.norm(w)
    dXr_dw = 0
    for i in range(X.shape[1]):
        X_ele = X[:,i:i+1]
        r3 = R[2:3,:]
        
        w_hat = np.dot(r3,X_ele)[0][0]+ t[2,0]
        s = (1-math.cos(theta))/(theta**2)
        if theta < 1e-5:
            X_rotated = X_ele + cross(w, X_ele)
            dXr_dw = skew(-X_ele)
        else:
            X_rotated = X_ele + Sinc(theta) * cross(w,  X_ele) + ((1 - np.cos(theta))/(theta**2)) * cross(w, cross(w,  X_ele))
            dXr_dw = Sinc(theta) * skew(-X_ele) + cross(w, X_ele) * dSinc(theta) * v_norm_de(w) + cross(w, cross(w, X_ele)) * ds_dtheta(theta) * v_norm_de(w) + s * (np.dot(skew(w), skew(-X_ele)) + skew(-cross(w, X_ele)))
        w_hat = float(X_rotated[2]) + t[2,0]
        dx_dXr = np.array([[1/w_hat, 0, -x_norm_inhom[0,i]/ w_hat],
                           [0, 1/w_hat, -x_norm_inhom[1,i]/ w_hat]])
        
        dx_dt = dx_dXr
        dX_dw =  np.dot(dx_dXr, dXr_dw)
        J_part = np.hstack((dX_dw, dx_dt))
        J[2*i:2*i+2,:] = J_part
    return J


In [466]:
a = np.array([[1,2,3]]).T
b = np.array([[4,5,6]]).T
print(np.cross(a.T, b.T).T)
print(np.cross(a,b,axis = 0))

[[-3]
 [ 6]
 [-3]]
[[-3]
 [ 6]
 [-3]]


In [606]:
def LM(P, x, X, K, max_iters, lam):
    # Inputs:
    #    P - initial estimate of camera pose
    #    x - 2D inliers
    #    X - 3D inliers
    #    K - camera calibration matrix 
    #    max_iters - maximum number of iterations
    #    lam - lambda parameter
    #
    # Output:
    #    P - Final camera pose obtained after convergence

    """your code here"""
    #print("P:{}".format(P))
    _iter = 0
    n = x.shape[1]
    x_hat = np.dot(K,np.dot(P,Homogenize(X)))
    error = (x-Dehomogenize(x_hat)).reshape((2*n,1), order = "F")
    #P = np.zeros((3,4))
    
    R,t = P[:3,:3], P[:,3:4]
    w,theta = Parameterize(R) 
    P_hat = np.array([[w[0][0],w[1][0],w[2][0],t[0][0],t[1][0],t[2][0]]]).T
    lamb = 0.001
    sigma_inv = np.linalg.inv(np.eye(2*n))
    min_Cost = ComputeCost(P, x, X, K)
    cost0 = min_Cost
    print("min_Cost:{}".format(min_Cost))
    while _iter < max_iters:
        J = Jacobian(R, w, t, X)
        #print("Jscobian:{}".format(J[:2,:]))
        normal_matrix = J.T@ sigma_inv @J + lam * np.eye(6)
        #print("shape of normal matrix:{}".format(normal_matrix.shape))
        normal_vect = J.T@ sigma_inv @ error
        delta = np.dot(np.linalg.inv(normal_matrix),normal_vect)
        #print("shape of delta:{}".format(delta.shape))
        P0_hat = P_hat + delta
        
        w1,w2,w3,t1,t2,t3 = P0_hat[:,0]
        w0 = np.array([[w1],[w2],[w3]])
        R0 = Deparameterize(w0)
        t0 = np.array([[t1,t2,t3]]).T
        
        P0 = np.hstack((R0,t0))
        cost0 = ComputeCost(P0, x, X, K)
        if cost0 < min_Cost:
            min_Cost = cost0
            P = P0
            lam *= 0.1
            _iter += 1
            P_hat = P0_hat
            print("i:{}, cost:{}".format(_iter,cost0))
        else:
            lam *= 10  
            if lam > 1e15:
                for _ in range(_iter,max_iters):
                    _iter += 1
                    print("i:{}, cost:{}".format(_iter,cost0))

    return P
 
# With the sample inliers...
# Start: 57.854356308
# End: 57.815478730

x0=np.loadtxt('hw3_points2D.txt').T
X0=np.loadtxt('hw3_points3D.txt').T
inliers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
x = x0[:,inliers]
X = X0[:,inliers]

# LM hyperparameters
lam = .001
max_iters = 100

tic = time.time()
P_LM = LM(P_linear, x, X, K, max_iters, lam)
w_LM,_ = Parameterize(P_LM[:,0:3])
toc = time.time()
time_total = toc-tic

# display the results
print('took %f secs'%time_total)
print('w_LM = ')
print(w_LM)
print('R_LM = ')
print(P_LM[:,0:3])
print('t_LM = ')
print(P_LM[:,-1])
      
      

min_Cost:386.49863248000986
i:1, cost:284.56392011568295
i:2, cost:218.4491673539792
i:3, cost:188.16039352035176
i:4, cost:187.10210107613636
i:5, cost:186.40226360264262
i:6, cost:186.06088694184638
i:7, cost:186.04646548817243
i:8, cost:186.03562883695486
i:9, cost:186.02837699400732
i:10, cost:186.02470996517582
i:11, cost:186.02462775627453
i:12, cost:186.02462775629365
i:13, cost:186.02462775629365
i:14, cost:186.02462775629365
i:15, cost:186.02462775629365
i:16, cost:186.02462775629365
i:17, cost:186.02462775629365
i:18, cost:186.02462775629365
i:19, cost:186.02462775629365
i:20, cost:186.02462775629365
i:21, cost:186.02462775629365
i:22, cost:186.02462775629365
i:23, cost:186.02462775629365
i:24, cost:186.02462775629365
i:25, cost:186.02462775629365
i:26, cost:186.02462775629365
i:27, cost:186.02462775629365
i:28, cost:186.02462775629365
i:29, cost:186.02462775629365
i:30, cost:186.02462775629365
i:31, cost:186.02462775629365
i:32, cost:186.02462775629365
i:33, cost:186.0246277